# lecure du zip et affichage des candles, 
# avec superposition de marques 
# suivant un calcul fait sur les candles

premier essai : on lit et on fait un petit traitement line par ligne a la lecture
pour l'affichage : pareil, on fait bourrin on génère un tableau de X et de Y a la mano et on le passe a plot


Voila le resultat 



In [ ]:
le meme programme avec un autre plot des detections mis dans zlibplot01

##### 

In [4]:
import zipfile

paire="EURUSD"
annee=2014
mois=4

nomfich="c:\\data\\HISTDATA_COM_ASCII_{0}_M1{1:4d}{2:02d}.zip".format(paire,annee,mois)
#nomfich2="c:\\data\\{0}_M1_{1:4d}{2:02d}.csv".format(paire,annee,mois)
nomfichshort="DAT_ASCII_{0}_M1_{1:4d}{2:02d}.csv".format(paire,annee,mois)

print nomfich

lstdata=[-1,0,0,0,0]


def scanfile(debut,longueur):
    fh = open(nomfich, 'rb')
    z = zipfile.ZipFile(fh) # classe lisant le zipdanzs le fichier ouvert
    
    minute =0
    
    quadvalues=[]
    
    with z.open(nomfichshort) as zz:
        for eachLine in zz:
            minute = minute+1
            if debut > minute :
                continue
            
            splitLine = eachLine.split(";")
            quadvalues = [minute,float(splitLine[1]),float(splitLine[4]),float(splitLine[2]),float(splitLine[3])]
        
            if minute == debut+longueur : #on a depasse la derniere valeur a lire : encore=0
                encore =0
                z.close()
                fh.close()
        
        
            yield encore,quadvalues
    
        #fin de la boucle car plus rien a lire
        encore =0
        z.close()
        fh.close()
        yield encore,[] #liste vide
           
        
        

#dit si une cande monte ou descend
def senscandle(data):
    
    if data[1]<data[2]:
        return 1   #deb < fin : ca monte
    if data[1]>data[2] :
        return 2  #deb > fin  ; ca descend
    return 0  #deb = fin


stackpast=[[-1,0,0,0,0]]




#detecte les retournements montee descente
def testrevertupdown(stack):
    #test un descente apres deux montees nettes
    cond1 = (stack[-2][2] > stack[-1][2]) #sortie acuelle inferieure  la sortie precedente
    cond2 = (stack[-3][2] > stack[-2][1]) and  (senscandle(stack[-2]) ==2)
    cond3 = (stack[-4][2] < stack[-3][1]) and (senscandle(stack[-3]) ==1) 
    cond4 = (stack[-5][2] < stack[-4][1]) and  (senscandle(stack[-4]) ==1)
    
    return cond1 , cond2 and cond3 and cond4 


def extractfeatures(stackdata):
    featureline=[]
    #sort les features d'avan le retournement
    featureline.append(stackdata[-2][4] - stackdata[-2][3]) #delta minmax
    featureline.append(stackdata[-2][1] - stackdata[-2][2]) #delta deb fin
    featureline.append(stackdata[-3][4] - stackdata[-3][3]) #delta minmax
    featureline.append(stackdata[-3][1] - stackdata[-3][2]) #delta deb fin
    featureline.append(stackdata[-4][4] - stackdata[-4][3]) #delta minmax
    featureline.append(stackdata[-4][1] - stackdata[-4][2]) #delta deb fin
    featureline.append(stackdata[-5][4] - stackdata[-5][3]) #delta minmax
    featureline.append(stackdata[-5][1] - stackdata[-5][2]) #delta deb fin
    
#    print "feature",stackdata,featureline
    return [i * 10000 for i in featureline] #muliply par 10000

#un petit essai d'analyse triviale pour prenre une decision triviale
# on met 1 si on sort rentre plus bas qu'on est sorti avant
# retourne ladifff qui vaut 1,0 ou 1 suivant que detection a ete faite ou non
# ok qui vaut 0 tant qu'on n'a âs eu assez de donnees
# learning qui contient une ligne de learnig matrice (feature  groupe)

def analyse(data):
    global lstdata
    ladiff =0
    ok=0
    learning=[] #init bidon
    
    stackpast.append(data[:]) #ajout de la derniere valeur ds stackpast
    
    if len(stackpast)==7 :
        ok=1
        del stackpast[0]  #stackpast ne doit plus s'allonger   
        yess,detected = testrevertupdown(stackpast)
        if detected:
            if yess:
                ladiff =1
            else:
                ladiff = -1
                    
        learning=extractfeatures(stackpast)  #extraction des features
        learning.extend([ladiff]) # categorie en fin de feature
        
    return ladiff,ok,learning    
    
    


datebar=[]
#entrybar=[]
#lowbar=[]
#highbar=[]
#outbar=[]


candlegraf=[]
analysetab=[]
learnmat=[]

minute =0
index=0

lescanfile = scanfile(100,1000)

while True:
    minute = minute+1
    encore,quadvalues = lescanfile.next()
    if encore ==0:
        break
            
    resu,ok,features = analyse(quadvalues)    
    if  resu<>0 :  #pile assez grosse pour faire les features : on genere les matrices
        learnmat.append(features) #matrice des features learning
        analysetab.append([resu, index]) #liste des events detectes pour la viualisation des detections 
    
    if ok:
        candlegraf.append(quadvalues) #matrice des valeurs brutes pour le graph cndles
        index= index+1

        
#icci on a la matrice de learning qui a son ensemble d'apprentissage et ses evenements en sortie
print "learning size = ",len(candlegraf)
print "positve size =",len([x for x in analysetab if x[0]==1])
print "negative size =",len([x for x in analysetab if x[0]==-1])


#print entrybar
import matplotlib.pyplot as ml

from matplotlib.finance import candlestick
import matplotlib.pyplot as plt

lafig = plt.figure()
ax1 = plt.subplot(1,1,1)
#ax2 = plt.subplot(2,1,2)


tabplot=[]
X=[]
Y=[]


#print tabplot

candlestick(ax1,candlegraf, width=.75, colorup='r', colordown='g')
#yscale = ax1.get_ybound()  #renvoie le min et max de l'afficahge
#print yscale

for i in range(len(analysetab)):
    if analysetab[i][0]==1 :
        indx = analysetab[i][1]
        X.append(candlegraf[indx][0])
        #on ajoute a y le doublet yscale multiplie par 1 (=lui meme) ou 0 (vide) 
        #c'est pas ce que je voulais faire , je voulais multipler chaque point de y (des 1) par une valeur
        #mais bon ca marche un peu. il va falloir passer tout ca en traiteent listes/matrices sinon c'est moche
        Y.append([candlegraf[indx][1],candlegraf[indx][2],candlegraf[indx][3],candlegraf[indx][4]])  

print len(X), len(Y)

ax1.plot(X,Y,'bo')

X=[]
Y=[]
for i in range(len(analysetab)):
    if analysetab[i][0]==-1 :
        indx = analysetab[i][1]
        X.append(candlegraf[indx][0])#on ajoute a y le doublet yscale multiplie par 1 (=lui meme) ou 0 (vide) 
        #c'est pas ce que je voulais faire , je voulais multipler chaque point de y (des 1) par une valeur
        #mais bon ca marche un peu. il va falloir passer tout ca en traiteent listes/matrices sinon c'est moche
        Y.append([candlegraf[indx][1],candlegraf[indx][2],candlegraf[indx][3],candlegraf[indx][4]])  

ax1.plot(X,Y,'yo')


plt.show()


#on a la matrice de features
#print learnmat

features = [x[0:-2] for x in learnmat]
categories = [x[-1] for x in learnmat]

#print features
#print categories

from sklearn import svm

clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(features, categories)

learnresu =[]

nbyesifyes = 0
nbnoifyes = 0
nbyesifno = 0


minute =0
nbchck=1000

#on rescane l'ensemble de check
lescanfile = scanfile(100,1000)

while True:
    minute = minute+1
    encore,quadvalues = lescanfile.next()
    if encore ==0:
        break

    resu,ok,features = analyse(quadvalues)     #calcul des features
    if  resu<>0 :  #on a un resultat
        valout = clf.predict(features[:-2])
        learnresu.append([resu,valout[0]]) #matrice valeur calc, valeur reelle            
    
    if ok:
        index= index+1

        
#analyse des resultats en comparant les resultats de la matrice des resultats

nbyesifyes=0
nbyesifno=0
nbnoifyes=0

for i in learnresu:
    valestim = i[1]
    valreelle = i[0]
    #print valreelle,valestim
    
    if valestim == 1 and valreelle==1:
        nbyesifyes = nbyesifyes+1
    
    if valestim == 1 and valreelle==-1:
        nbyesifno = nbyesifno+1
  
    if valestim == -1 and valreelle==1:
        nbnoifyes = nbnoifyes+1
    
print '%correct',float(nbyesifyes)/float(nbyesifyes+nbyesifno) *100.0
print '%detected' , float(nbyesifyes)/float(nbyesifyes+nbnoifyes) *100.0 


z.close()

c:\data\HISTDATA_COM_ASCII_EURUSD_M1201404.zip


IOError: [Errno 2] No such file or directory: 'c:\\data\\HISTDATA_COM_ASCII_EURUSD_M1201404.zip'

In [47]:
learnresu



[[-1, -1],
 [1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [1, -1],
 [-1, -1],
 [1, 1],
 [1, -1],
 [1, -1],
 [-1, -1],
 [1, 1],
 [1, -1],
 [1, 1],
 [-1, 1],
 [1, -1],
 [-1, 1],
 [-1, 1],
 [-1, 1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, 1],
 [-1, 1],
 [1, -1],
 [-1, -1],
 [1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [1, -1],
 [1, 1],
 [-1, 1],
 [-1, 1],
 [-1, -1],
 [-1, -1],
 [-1, 1],
 [1, -1],
 [1, -1],
 [-1, -1],
 [1, -1],
 [1, -1],
 [-1, -1],
 [1, -1],
 [1, -1],
 [-1, 1],
 [1, 1],
 [-1, 1],
 [1, 1],
 [1, 1],
 [1, 1],
 [1, -1],
 [1, -1],
 [1, -1],
 [1, -1],
 [1, -1],
 [-1, -1],
 [1, -1],
 [-1, -1],
 [-1, -1],
 [1, -1],
 [1, -1],
 [1, 1],
 [1, -1],
 [-1, 1],
 [1, -1],
 [1, 1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [1, -1],
 [1, -1],
 [1, -1],
 [1, -1],
 [-1, -1],
 [1, -1],
 [-1, -1],
 [1, -1],
 [1, 1],
 [-1, 1],
 [-1, 1],
 [-1, -1],
 [1, 1],
 [-1, -1],
 [-1, 1],
 [1, -1],
 [1, -1],
 [-1, -1],
 [-1, -1],
 [1, -1],
 [-1, 1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [1, 1]